In [78]:
from opentree import OT
import os

In [79]:
!ls ../ott3.2

forwards.tsv  README.html  synonyms.tsv  taxonomy.tsv  version.txt


In [80]:
taxonomy_file = "../ott3.2/taxonomy.tsv"
assert os.path.exists(taxonomy_file)

In [81]:
fam_dict = {}
for lin in open(taxonomy_file):
        lii=lin.split('\t|\t')
        if len(lii[2].split(' ')) > 1:
            pass
        elif lii[2].endswith("aceae"):
            fam_dict[lii[2]]=lii
        elif lii[2].endswith("idae"):
            fam_dict[lii[2]]=lii

In [83]:
maybe_not_fams = set()
for fam in fam_dict:
    if fam_dict[fam][3] == 'family':
        pass
    else:
        maybe_not_fams.add(fam)

In [84]:
len(maybe_not_fams)

163

In [85]:
#fams_ott_id = {}

In [86]:
for fam in fam_dict:
    if fam not in fams_ott_id:
        ott_id = fam_dict[fam][0]
        fams_ott_id[fam] = OT.synth_node_info(ott_id = ott_id)

In [87]:
node_ids = {}
fams_in_tree = set()
not_in_tree = set()
for fam in fams_ott_id:
    nid = fams_in_tree[fam].response_dict.get('node_id')
    if nid:
        if nid not in node_ids:
            node_ids[nid] = set()
        node_ids[nid].add(fam)
        fams_in_tree.add(fam)
    else:
        not_in_tree.add(fam)

TypeError: 'set' object is not subscriptable

In [ ]:
len(fams_in_tree)

In [ ]:
fi = open("families_not_in_tree.csv",'w')
fi.write('name\t ottid, sources, flags\n')
for fam in not_in_tree:
    fi.write("{}\t{}\t{}\t{}\n".format(fam,fam_dict[fam][0],fam_dict[fam][-4],fam_dict[fam][-2]))

fi.close()

In [ ]:
fi = open("families_in_tree.csv",'w')
fi.write('name\t ottid, sources\n')
for fam in fams_in_tree:
    fi.write("{}\t{}\t{}\t{}\n".format(fam,fam_dict[fam][0],fam_dict[fam][-4]))

fi.close()

In [ ]:
len(node_ids)

In [54]:
tree = OT.synth_induced_tree(node_ids = list(node_ids.keys()), label_format="name")

In [51]:
tree.response_dict

{'broken': {},
 'newick': "(((((((((((((((((((((((((((((((((((((((((((Elaeocarpaceae_ott649007,(Cunoniaceae_ott782239,Cephalotaceae_ott782231)mrcaott5195ott782242)mrcaott2ott5195,Brunelliaceae_ott1053057)mrcaott2ott520300,(Oxalidaceae_ott372826,Connaraceae_ott766272)mrcaott10190ott380061)mrcaott2ott10190,Huaceae_ott36015)Oxalidales_ott431493,((((((((mrcaott24237ott109257)mrcaott345ott24237,Ochnaceae_ott412944)mrcaott345ott38722,Caryocaraceae_ott436442)mrcaott345ott177705,((((Linaceae_ott1000257,Centroplacaceae_ott928858)mrcaott18689ott148601,Pandaceae_ott552732)mrcaott18689ott31619,(((Dichapetalaceae_ott1035663,Trigoniaceae_ott107577)mrcaott24242ott51202,(Chrysobalanaceae_ott1009819,Euphroniaceae_ott2420)mrcaott101136ott669064)mrcaott24242ott101136,Balanopaceae_ott783915)mrcaott24242ott565495)mrcaott18689ott24242,((Erythroxylaceae_ott567817,Rhizophoraceae_ott655809)mrcaott31615ott47043,Ctenolophonaceae_ott387173)mrcaott31615ott387171)mrcaott18689ott31615)mrcaott345ott18689,(Putranjivac

KeyboardInterrupt: 

In [56]:
tree.tree

In [57]:
tree.tree.write(path="allfam.tre",schema="newick")

In [62]:
def remove_problem_characters(tree, prob_char = "():#", replace_w = '?'):
    problem_characters = set(prob_char)
    for node in tree:
        if node.taxon:
            for char in problem_characters:
                node.taxon.label = node.taxon.label.replace(char,replace_w)
        elif node.label:
            for char in problem_characters:
                node.label = node.label.replace(char,replace_w)
    return None

In [63]:
import copy
local_tree = copy.deepcopy(tree.tree)

remove_problem_characters(local_tree)
local_tree.write(path="allfam.tre",schema="newick")

In [66]:
# Don't forget to cite your friendly phylogeneticists!
studies = tree.response_dict['supporting_studies']
cites = OT.get_citations(studies) #this can be a bit slow
fi = open("all_famcites.txt","w")
fi.write(cites)
fi.close()

In [ ]:
len(studies)